### 基准模型

为了和卷积神经网络分类器进行对比。这里学习得到逻辑回归和SVM模型，计算测试集分类准确率。

In [12]:
from __future__ import print_function
from six.moves import cPickle as pickle

pickle_file = 'svhn_single.pickle'

with open(pickle_file, 'rb') as f:
    dataset = pickle.load(f)
    train_dataset = dataset['train_dataset']
    train_labels = dataset['train_labels']
    valid_dataset = dataset['valid_dataset']
    valid_labels = dataset['valid_labels']
    test_dataset = dataset['test_dataset'][:10000]
    test_labels = dataset['test_labels'][:10000]
    del dataset
    
print('Training dataset shape: ', train_dataset.shape)
print('Training labels shape: ', train_labels.shape)
print('Validing dataset shape: ', valid_dataset.shape)
print('Validing labels shape: ', valid_labels.shape)
print('Testing dataset shape: ', test_dataset.shape)
print('Testing labels shape: ', test_labels.shape)

Training dataset shape:  (72525, 32, 32)
Training labels shape:  (72525,)
Validing dataset shape:  (732, 32, 32)
Validing labels shape:  (732,)
Testing dataset shape:  (10000, 32, 32)
Testing labels shape:  (10000,)


### 将数据修改成适合sklearn的格式

逻辑回归分类器和SVM接受的输入是一维向量，需要将二维的图片数据矢量化为一维。  
在矢量化的过程中图片的空间信息很大程度上被丢失。卷积神经网络可以直接处理三维的图片数据，很好地保留了图片的空间信息，事实上是卷积神经网络一个极大的优势。

In [14]:
IMAGE_SIZE = 32
NUM_LABELS = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, IMAGE_SIZE*IMAGE_SIZE*num_channels)).astype(np.float32)
    #labels = (np.array([10,1,2,3,4,5,6,7,8,9]) == labels.reshape(-1, 1)).astype(np.float32) # one-hot encoding
    labels[labels==10] = 0
    return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)

print('Training dataset shape: ', train_dataset.shape)
print('Training labels shape: ', train_labels.shape)
print('Validing dataset shape: ', valid_dataset.shape)
print('Validing labels shape: ', valid_labels.shape)
print('Testing dataset shape: ', test_dataset.shape)
print('Testing labels shape: ', test_labels.shape)

Training dataset shape:  (72525, 1024)
Training labels shape:  (72525,)
Validing dataset shape:  (732, 1024)
Validing labels shape:  (732,)
Testing dataset shape:  (10000, 1024)
Testing labels shape:  (10000,)


### 逻辑回归分类器

In [15]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression()
classifier.fit(train_dataset, train_labels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

### SVM分类器

In [33]:
from sklearn.linear_model import SGDClassifier

classifier = SGDClassifier(loss='hinge')
classifier.fit(train_dataset, train_labels)

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

### 分类结果

逻辑回归分类器测试集分类准确率为0.2364。SVM分类器测试集分类准确率为0.1687。  
数据集的类别有10类，对应于10个数字。使用一个随机的分类器，即完全不使用图片的信息，随机确定一个类别，分类的准确率会有0.1。因此可见逻辑回归分类机和SVM分类器在此问题上的效果有多差。

In [34]:
from sklearn.metrics import accuracy_score

test_prediction = classifier.predict(test_dataset)
print('Test accuracy', accuracy_score(test_labels, test_prediction))

Test accuracy 0.1687


In [35]:
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(test_labels, test_prediction))
print('Confusion matrix')
print(confusion_matrix(test_labels, test_prediction))

             precision    recall  f1-score   support

          0       0.29      0.10      0.14       684
          1       0.28      0.32      0.30      1968
          2       0.28      0.14      0.18      1590
          3       0.20      0.12      0.15      1104
          4       0.16      0.22      0.18       964
          5       0.22      0.13      0.17       909
          6       0.11      0.04      0.05       773
          7       0.12      0.06      0.08       767
          8       0.06      0.33      0.11       621
          9       0.10      0.07      0.08       620

avg / total       0.20      0.17      0.17     10000

Confusion matrix
[[ 65 158  49  16  43  18  11  15 268  41]
 [ 26 629 107 133 373  88  31  30 472  79]
 [ 19 365 215  55 242  26  27  69 505  67]
 [ 18 261  63 129 120  53  40  44 327  49]
 [  9 196  46  45 209  28  22  83 286  40]
 [ 13 156  38  78  71 121  53  22 313  44]
 [ 31 160  38  36  70  86  28  14 263  47]
 [ 16 135 108  40  83  27  11  43 276  28]
